## House Prices Competition level2

In [4]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [8]:
# Load the data, and separate the target
iowa_file_path = 'input/train.csv'
home_data = pd.read_csv(iowa_file_path)
y = home_data.SalePrice

In [9]:
features = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']

In [10]:
X = home_data[features]
X.head()

,LotArea,YearBuilt,1stFlrSF,2ndFlrSF,FullBath,BedroomAbvGr,TotRmsAbvGrd
0,8450,2003,856,854,2,3,8
1,9600,1976,1262,0,2,3,6
2,11250,2001,920,866,2,3,6
3,9550,1915,961,756,1,3,7
4,14260,2000,1145,1053,2,4,9


In [11]:
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

In [12]:
# Define a random forest model
model = RandomForestRegressor(random_state=1)
model.fit(train_X, train_y)
predictions = model.predict(val_X)
rf_val_mae = mean_absolute_error(predictions, val_y)

print("Validation MAE for Random Forest Model: {:,.0f}".format(rf_val_mae))

Validation MAE for Random Forest Model: 21,857


In [13]:
def generate_sub(X,y):

    rf_model_on_full_data = RandomForestRegressor()
    rf_model_on_full_data.fit(X,y)

    test_data_path = 'input/test.csv'

    # read test data file using pandas
    test_data = pd.read_csv(test_data_path)

    # create test_X which comes from test_data but includes only the columns you used for prediction.
    # The list of columns is stored in a variable called features
    test_X = test_data[features]

    # make predictions which we will submit. 
    test_preds = rf_model_on_full_data.predict(test_X)

    ## create my submission file
    output = pd.DataFrame({'Id': test_data.Id,
                        'SalePrice': test_preds})
    output.to_csv('out/submission.csv', index=False)